# <a href="https://colab.research.google.com/github/forrestshort/electrodynamics/blob/master/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Midterm Project

Forrest Short

## Helmholtz Coil

This project is a visualization of the magnetic field produced by a Helmholtz coil



## Load Packages


In [1]:
try:
    # Import gmsh library for generating meshes.
    import gmsh
except ImportError:
    # If it is not available, install it.  Then import it.
    !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"
    import gmsh
    
try:
    # Import FEniCSx libraries for finite element analysis.
    import dolfinx
except ImportError:
    # If they are not found, install them.  Then import them.
    !wget "https://fem-on-colab.github.io/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx
    
try:
    # Import multiphenicsx, mainly for plotting.
    import multiphenicsx
except ImportError:
    # If they are not found, install them.
    !pip3 install "multiphenicsx@git+https://github.com/multiphenics/multiphenicsx.git@8b97b4e"
    import multiphenicsx
    
import dolfinx.fem
import dolfinx.io
import gmsh
import mpi4py.MPI
import numpy as np
import petsc4py.PETSc
import ufl
from dolfinx.io import gmshio
#import multiphenicsx.fem
import multiphenicsx.io

## Simulation Inputs


In [2]:
# Mesh Dimension
dim = 2

# Radius of background disk
r_background = 5

# µ background 
mu_background = 1

# µ wires
mu_wire = 6

# Radius of Coils
r_wire = 0.1
r_helm = 0.13
r_wider = 0.17

# Number of Coils.
N = 12

# Current density in each coil.
J0 = 5

# Run All

In [ ]:
# Create a model.
gmsh.initialize()
gmsh.model.add("mesh")

# Surface Tags
inner_tag = 2
outer_tag = 2 + N
background_surfaces = []
other_surfaces = []

# Background
background = gmsh.model.occ.addDisk(0, 0, 0, r_background, r_background)
gmsh.model.occ.synchronize()


wires_in = []
wires_out = []

wires_out.append( (2, gmsh.model.occ.addDisk( 1, -0.5, 0, r_wire, r_wire)) )
wires_out.append( (2, gmsh.model.occ.addDisk( 1,  0.5, 0, r_wire, r_wire)) )

wires_in.append( (2, gmsh.model.occ.addDisk(-1, -0.5, 0, r_wire, r_wire)) )
wires_in.append( (2, gmsh.model.occ.addDisk(-1,  0.5, 0, r_wire, r_wire)) )

gmsh.model.occ.synchronize()


# Resolve Boundaries
all_surfaces = []
all_surfaces.extend(wires_in)
all_surfaces.extend(wires_out)
whole_domain = gmsh.model.occ.fragment([(2, background)], all_surfaces)

gmsh.model.occ.synchronize()


# Assign Tags to Surfaces
for domain in whole_domain[0]:
    center = gmsh.model.occ.getCenterOfMass(domain[0], domain[1])
    mass = gmsh.model.occ.getMass(domain[0], domain[1])


    if np.allclose(center, [0, 0, 0]):
        background_surfaces.append(domain[1])

    elif np.isclose(center[0], -1):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], inner_tag)
        inner_tag +=1
        other_surfaces.append(domain)

    elif np.isclose(center[0], +1):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], outer_tag)
        outer_tag +=1
        other_surfaces.append(domain)



gmsh.model.addPhysicalGroup(2, background_surfaces, tag=0)



# Create a mesh for this system.
gmsh.model.mesh.generate(dim)

# Bring the mesh into FEniCSx.
mesh, subdomains, boundaries = dolfinx.io.gmshio.model_to_mesh(
    gmsh.model, comm=mpi4py.MPI.COMM_WORLD, rank=0, gdim=2)

gmsh.finalize()

Q = dolfinx.fem.FunctionSpace(mesh, ("DG", 0))

# Get the list of materials.
material_tags = np.unique(subdomains.values)

# Define functions for current density and magnetic permeability.
mu = dolfinx.fem.Function(Q)
J = dolfinx.fem.Function(Q)

# Only some regions carry current. Initialize all current densities to zero.
J.x.array[:] = 0.0

# Now, cycle over all objects and assign material properties. 
for tag in material_tags:
    cells = subdomains.find(tag)
    
    # Set values for magnetic permeability.
    if tag == 0:
        # Vacuum
        mu_ = mu_background
    else:
        # Wire
        mu_ = mu_wire

    mu.x.array[cells] = np.full_like(cells, mu_, dtype=petsc4py.PETSc.ScalarType)
    
    # Set nonzero current densities.
    if tag in range(2, 2+N):
        J.x.array[cells] = np.full_like(cells, J0, dtype=petsc4py.PETSc.ScalarType)
    elif tag in range(2+N, 2*N + 2):
        J.x.array[cells] = np.full_like(cells, -J0, dtype=petsc4py.PETSc.ScalarType)
        
## Set up the finite element problem.

# Define trial and test functions.
V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 2))

# Define the trial and test functions.
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Create a function to store the solution.
# This is the vector potential.  A_x = A_y = 0.
A_z1 = dolfinx.fem.Function(V)

# Identify the domain and boundary.
D = mesh.topology.dim
Omega = dolfinx.mesh.locate_entities_boundary(mesh, D-1, lambda x: np.full(x.shape[1], True))
dOmega = dolfinx.fem.locate_dofs_topological(V, D-1, Omega)

# Force the potential to vanish on the boundary.
bc = dolfinx.fem.dirichletbc(petsc4py.PETSc.ScalarType(0), dOmega, V)

# Define the Poisson equation we are trying to solve.
a = (1 / mu) * ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = 4 * ufl.pi * J * v * ufl.dx

# Define the problem.
problem = dolfinx.fem.petsc.LinearProblem(a, L, u=A_z1, bcs=[bc])

# Solve the problem.
problem.solve()


# Compute the magnetic field.
W = dolfinx.fem.VectorFunctionSpace(mesh, ("CG", 2))
B1 = dolfinx.fem.Function(W)
B_expr = dolfinx.fem.Expression(ufl.as_vector((A_z1.dx(1), -A_z1.dx(0))), W.element.interpolation_points())
B1.interpolate(B_expr)


# Create a model.
gmsh.initialize()
gmsh.model.add("mesh")

# Define the system: a large disk.
background = gmsh.model.occ.addDisk(0, 0, 0, r_background, r_background)
gmsh.model.occ.synchronize()

inner_tag = 2
outer_tag = 2 + N
background_surfaces = []
other_surfaces = []

wires_in = []
wires_out = []

wires_out.append( (2, gmsh.model.occ.addDisk( 1, -0.5, 0, r_wire, r_wire)) )
wires_out.append( (2, gmsh.model.occ.addDisk( 1,  0, 0, r_wire, r_wire)) )
wires_out.append( (2, gmsh.model.occ.addDisk( 1,  0.5, 0, r_wire, r_wire)) )

wires_in.append( (2, gmsh.model.occ.addDisk(-1, -0.5, 0, r_wire, r_wire)) )
wires_in.append( (2, gmsh.model.occ.addDisk(-1,  0, 0, r_wire, r_wire)) )
wires_in.append( (2, gmsh.model.occ.addDisk(-1,  0.5, 0, r_wire, r_wire)) )

# Update the model.
gmsh.model.occ.synchronize()

# Resolve the boundaries of the wires and ring in the background domain.

all_surfaces = []
all_surfaces.extend(wires_in)
all_surfaces.extend(wires_out)
whole_domain = gmsh.model.occ.fragment([(2, background)], all_surfaces)

# Update the model.
gmsh.model.occ.synchronize()


inner_tag = 2
outer_tag = 2 + N
background_surfaces = []
other_surfaces = []


for domain in whole_domain[0]:
    center = gmsh.model.occ.getCenterOfMass(domain[0], domain[1])
    mass = gmsh.model.occ.getMass(domain[0], domain[1])

    
    # Identify the background circle by its center of mass
    if np.allclose(center, [0, 0, 0]):
        background_surfaces.append(domain[1])

    # Identify the inner wires by their centers of mass.
    elif np.isclose(center[0], -1):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], inner_tag)
        inner_tag +=1
        other_surfaces.append(domain)

    elif np.isclose(center[0], -1.2):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], inner_tag)
        inner_tag +=1
        other_surfaces.append(domain)

    # Identify the outer wires by their center of mass.
    elif np.isclose(center[0], +1):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], outer_tag)
        outer_tag +=1
        other_surfaces.append(domain)

    elif np.isclose(center[0], +1.2):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], outer_tag)
        outer_tag +=1
        other_surfaces.append(domain)

# Add marker for the vacuum.
gmsh.model.addPhysicalGroup(2, background_surfaces, tag=0)



# Create a mesh for this system.
gmsh.model.mesh.generate(dim)

# Bring the mesh into FEniCSx.
mesh, subdomains, boundaries = dolfinx.io.gmshio.model_to_mesh(
    gmsh.model, comm=mpi4py.MPI.COMM_WORLD, rank=0, gdim=2)

gmsh.finalize()

# This loop will assign material properties to each cell in our model.
# In this case, it is the relative magnetic permeability and current density.

# Define a simple function space for properties.
Q = dolfinx.fem.FunctionSpace(mesh, ("DG", 0))

# Get the list of materials.
material_tags = np.unique(subdomains.values)

# Define functions for current density and magnetic permeability.
mu = dolfinx.fem.Function(Q)
J = dolfinx.fem.Function(Q)

# Only some regions carry current. Initialize all current densities to zero.
J.x.array[:] = 0.0

# Now, cycle over all objects and assign material properties. 
for tag in material_tags:
    cells = subdomains.find(tag)
    
    # Set values for magnetic permeability.
    if tag == 0:
        # Vacuum
        mu_ = mu_background
    elif tag == 1:
        # Ring
        mu_ = mu_ring
    else:
        # Wire
        mu_ = mu_wire

    mu.x.array[cells] = np.full_like(cells, mu_, dtype=petsc4py.PETSc.ScalarType)
    
    # Set nonzero current densities.
    if tag in range(2, 2+N):
        J.x.array[cells] = np.full_like(cells, J0, dtype=petsc4py.PETSc.ScalarType)
    elif tag in range(2+N, 2*N + 2):
        J.x.array[cells] = np.full_like(cells, -J0, dtype=petsc4py.PETSc.ScalarType)
        
## Set up the finite element problem.

# Define trial and test functions.
V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 2))

# Define the trial and test functions.
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Create a function to store the solution.
# This is the vector potential.  A_x = A_y = 0.
A_z2 = dolfinx.fem.Function(V)

# Identify the domain and boundary.
D = mesh.topology.dim
Omega = dolfinx.mesh.locate_entities_boundary(mesh, D-1, lambda x: np.full(x.shape[1], True))
dOmega = dolfinx.fem.locate_dofs_topological(V, D-1, Omega)

# Force the potential to vanish on the boundary.
bc = dolfinx.fem.dirichletbc(petsc4py.PETSc.ScalarType(0), dOmega, V)

# Define the Poisson equation we are trying to solve.
a = (1 / mu) * ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = 4 * ufl.pi * J * v * ufl.dx

# Define the problem.
problem = dolfinx.fem.petsc.LinearProblem(a, L, u=A_z2, bcs=[bc])

# Solve the problem.
problem.solve()


# Compute the magnetic field.
W = dolfinx.fem.VectorFunctionSpace(mesh, ("CG", 2))
B2 = dolfinx.fem.Function(W)
B_expr = dolfinx.fem.Expression(ufl.as_vector((A_z2.dx(1), -A_z2.dx(0))), W.element.interpolation_points())
B2.interpolate(B_expr)

# Create a model.
gmsh.initialize()
gmsh.model.add("mesh")

# Define the system: a large disk.
background = gmsh.model.occ.addDisk(0, 0, 0, r_background, r_background)
gmsh.model.occ.synchronize()


wires_in = []
wires_out = []

# These seemingly weirdly specific values come from James Clerk Maxwell 
# himself and were outlined in the 1873 book "A Treatise on Electricity and Magentism"
# The design parameters are on pages 320-321
# https://archive.org/details/electricandmag02maxwrich/page/n351/mode/2up?view=theater

wires_out.append( (2, gmsh.model.occ.addDisk( 0.9, -0.786, 0, r_wire, r_wire)) )
wires_out.append( (2, gmsh.model.occ.addDisk( 1.2,  0, 0, r_helm, r_helm)) )
wires_out.append( (2, gmsh.model.occ.addDisk( 0.9,  0.786, 0, r_wire, r_wire)) )

wires_in.append( (2, gmsh.model.occ.addDisk(-0.9, -0.786, 0, r_wire, r_wire)) )
wires_in.append( (2, gmsh.model.occ.addDisk(-1.2,  0, 0, r_helm, r_helm)) )
wires_in.append( (2, gmsh.model.occ.addDisk(-0.9,  0.786, 0, r_wire, r_wire)) )

# Update the model.
gmsh.model.occ.synchronize()

# Resolve the boundaries of the wires and ring in the background domain.

all_surfaces = []
all_surfaces.extend(wires_in)
all_surfaces.extend(wires_out)
whole_domain = gmsh.model.occ.fragment([(2, background)], all_surfaces)

# Update the model.
gmsh.model.occ.synchronize()

# Create physical markers for each object.
# Use the following markers:
# - Vacuum: 0
# - Ring: 1
# - Inner wires: $[2,3,\dots,N+1]$
# - Outer wires: $[N+2,\dots, 2\cdot N+1]$
inner_tag = 2
outer_tag = 2 + N
background_surfaces = []
other_surfaces = []

# Gmsh can compute the mass of objects and the location of their
# centers of mass.  This loop uses these properties to determine
# which object to associate grid points with.
# 
# We will use these tags to define material properties later.
for domain in whole_domain[0]:
    center = gmsh.model.occ.getCenterOfMass(domain[0], domain[1])
    mass = gmsh.model.occ.getMass(domain[0], domain[1])

    
    # Identify the background circle by its center of mass
    if np.allclose(center, [0, 0, 0]):
        background_surfaces.append(domain[1])

    # Identify the inner wires by their centers of mass.
    elif np.isclose(center[0], -0.9):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], inner_tag)
        inner_tag +=1
        other_surfaces.append(domain)

    elif np.isclose(center[0], -1.2):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], inner_tag)
        inner_tag +=1
        other_surfaces.append(domain)

    # Identify the outer wires by their center of mass.
    elif np.isclose(center[0], +0.9):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], outer_tag)
        outer_tag +=1
        other_surfaces.append(domain)

    elif np.isclose(center[0], +1.2):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], outer_tag)
        outer_tag +=1
        other_surfaces.append(domain)

# Add marker for the vacuum.
gmsh.model.addPhysicalGroup(2, background_surfaces, tag=0)



# Create a mesh for this system.
gmsh.model.mesh.generate(dim)

# Bring the mesh into FEniCSx.
mesh, subdomains, boundaries = dolfinx.io.gmshio.model_to_mesh(
    gmsh.model, comm=mpi4py.MPI.COMM_WORLD, rank=0, gdim=2)

gmsh.finalize()

# This loop will assign material properties to each cell in our model.
# In this case, it is the relative magnetic permeability and current density.

# Define a simple function space for properties.
Q = dolfinx.fem.FunctionSpace(mesh, ("DG", 0))

# Get the list of materials.
material_tags = np.unique(subdomains.values)

# Define functions for current density and magnetic permeability.
mu = dolfinx.fem.Function(Q)
J = dolfinx.fem.Function(Q)

# Only some regions carry current. Initialize all current densities to zero.
J.x.array[:] = 0.0

# Now, cycle over all objects and assign material properties. 
for tag in material_tags:
    cells = subdomains.find(tag)
    
    # Set values for magnetic permeability.
    if tag == 0:
        # Vacuum
        mu_ = mu_background

    else:
        # Wire
        mu_ = mu_wire

    mu.x.array[cells] = np.full_like(cells, mu_, dtype=petsc4py.PETSc.ScalarType)
    
    # Set nonzero current densities.
    if tag in range(2, 2+N):
        J.x.array[cells] = np.full_like(cells, J0, dtype=petsc4py.PETSc.ScalarType)
    elif tag in range(2+N, 2*N + 2):
        J.x.array[cells] = np.full_like(cells, -J0, dtype=petsc4py.PETSc.ScalarType)
        
## Set up the finite element problem.

# Define trial and test functions.
V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 2))

# Define the trial and test functions.
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Create a function to store the solution.
# This is the vector potential.  A_x = A_y = 0.
A_z3 = dolfinx.fem.Function(V)

# Identify the domain and boundary.
D = mesh.topology.dim
Omega = dolfinx.mesh.locate_entities_boundary(mesh, D-1, lambda x: np.full(x.shape[1], True))
dOmega = dolfinx.fem.locate_dofs_topological(V, D-1, Omega)

# Force the potential to vanish on the boundary.
bc = dolfinx.fem.dirichletbc(petsc4py.PETSc.ScalarType(0), dOmega, V)

# Define the Poisson equation we are trying to solve.
a = (1 / mu) * ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = 4 * ufl.pi * J * v * ufl.dx

# Define the problem.
problem = dolfinx.fem.petsc.LinearProblem(a, L, u=A_z3, bcs=[bc])

# Solve the problem.
problem.solve()


# Compute the magnetic field.
W = dolfinx.fem.VectorFunctionSpace(mesh, ("CG", 2))
B3 = dolfinx.fem.Function(W)
B_expr = dolfinx.fem.Expression(ufl.as_vector((A_z3.dx(1), -A_z3.dx(0))), W.element.interpolation_points())
B3.interpolate(B_expr)

# Create a model.
gmsh.initialize()
gmsh.model.add("mesh")

# Define the system: a large disk.
background = gmsh.model.occ.addDisk(0, 0, 0, r_background, r_background)
gmsh.model.occ.synchronize()

inner_tag = 2
outer_tag = 2 + N
background_surfaces = []
other_surfaces = []

wires_in = []
wires_out = []

wires_out.append( (2, gmsh.model.occ.addDisk( 1, -0.5, 0, r_wire, r_wire)))
wires_out.append( (2, gmsh.model.occ.addDisk( 1.2,  -0.25, 0, r_helm, r_helm)))
wires_out.append( (2, gmsh.model.occ.addDisk( 1.4,  0, 0, r_wider, r_wider)))
wires_out.append( (2, gmsh.model.occ.addDisk( 1.2,  0.25, 0, r_helm, r_helm)))
wires_out.append( (2, gmsh.model.occ.addDisk( 1,  0.5, 0, r_wire, r_wire)))

wires_in.append( (2, gmsh.model.occ.addDisk(-1, -0.5, 0, r_wire, r_wire)))
wires_in.append( (2, gmsh.model.occ.addDisk(-1.2,  -0.25, 0, r_helm, r_helm)))
wires_in.append( (2, gmsh.model.occ.addDisk(-1.4,  0, 0, r_wider, r_wider)))
wires_in.append( (2, gmsh.model.occ.addDisk(-1.2,  0.25, 0, r_helm, r_helm)))
wires_in.append( (2, gmsh.model.occ.addDisk(-1,  0.5, 0, r_wire, r_wire)))

# Update the model.
gmsh.model.occ.synchronize()

# Resolve the boundaries of the wires and ring in the background domain.

all_surfaces = []
all_surfaces.extend(wires_in)
all_surfaces.extend(wires_out)
whole_domain = gmsh.model.occ.fragment([(2, background)], all_surfaces)

# Update the model.
gmsh.model.occ.synchronize()


for domain in whole_domain[0]:
    center = gmsh.model.occ.getCenterOfMass(domain[0], domain[1])
    mass = gmsh.model.occ.getMass(domain[0], domain[1])

    
    # Identify the background circle by its center of mass
    if np.allclose(center, [0, 0, 0]):
        background_surfaces.append(domain[1])

    # Identify the inner wires by their centers of mass.
    elif np.isclose(center[0], -1):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], inner_tag)
        inner_tag +=1
        other_surfaces.append(domain)

    elif np.isclose(center[0], -1.2):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], inner_tag)
        inner_tag +=1
        other_surfaces.append(domain)

    elif np.isclose(center[0], -1.4):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], inner_tag)
        inner_tag +=1
        other_surfaces.append(domain)

    # Identify the outer wires by their center of mass.
    elif np.isclose(center[0], +1):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], outer_tag)
        outer_tag +=1
        other_surfaces.append(domain)

    elif np.isclose(center[0], +1.2):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], outer_tag)
        outer_tag +=1
        other_surfaces.append(domain)


    elif np.isclose(center[0], +1.4):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], outer_tag)
        outer_tag +=1
        other_surfaces.append(domain)



        




# Add marker for the vacuum.
gmsh.model.addPhysicalGroup(2, background_surfaces, tag=0)



# Create a mesh for this system.
gmsh.model.mesh.generate(dim)

# Bring the mesh into FEniCSx.
mesh, subdomains, boundaries = dolfinx.io.gmshio.model_to_mesh(
    gmsh.model, comm=mpi4py.MPI.COMM_WORLD, rank=0, gdim=2)

gmsh.finalize()

# This loop will assign material properties to each cell in our model.
# In this case, it is the relative magnetic permeability and current density.

# Define a simple function space for properties.
Q = dolfinx.fem.FunctionSpace(mesh, ("DG", 0))

# Get the list of materials.
material_tags = np.unique(subdomains.values)

# Define functions for current density and magnetic permeability.
mu = dolfinx.fem.Function(Q)
J = dolfinx.fem.Function(Q)

# Only some regions carry current. Initialize all current densities to zero.
J.x.array[:] = 0.0

# Now, cycle over all objects and assign material properties. 
for tag in material_tags:
    cells = subdomains.find(tag)
    
    # Set values for magnetic permeability.
    if tag == 0:
        # Vacuum
        mu_ = mu_background

    else:
        # Wire
        mu_ = mu_wire

    mu.x.array[cells] = np.full_like(cells, mu_, dtype=petsc4py.PETSc.ScalarType)
    
    # Set nonzero current densities.
    if tag in range(2, 2+N):
        J.x.array[cells] = np.full_like(cells, J0, dtype=petsc4py.PETSc.ScalarType)
    elif tag in range(2+N, 2*N + 2):
        J.x.array[cells] = np.full_like(cells, -J0, dtype=petsc4py.PETSc.ScalarType)
        
## Set up the finite element problem.

# Define trial and test functions.
V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 2))

# Define the trial and test functions.
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Create a function to store the solution.
# This is the vector potential.  A_x = A_y = 0.
A_z4 = dolfinx.fem.Function(V)

# Identify the domain and boundary.
D = mesh.topology.dim
Omega = dolfinx.mesh.locate_entities_boundary(mesh, D-1, lambda x: np.full(x.shape[1], True))
dOmega = dolfinx.fem.locate_dofs_topological(V, D-1, Omega)

# Force the potential to vanish on the boundary.
bc = dolfinx.fem.dirichletbc(petsc4py.PETSc.ScalarType(0), dOmega, V)

# Define the Poisson equation we are trying to solve.
a = (1 / mu) * ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = 4 * ufl.pi * J * v * ufl.dx

# Define the problem.
problem = dolfinx.fem.petsc.LinearProblem(a, L, u=A_z4, bcs=[bc])

# Solve the problem.
problem.solve()


# Compute the magnetic field.
W = dolfinx.fem.VectorFunctionSpace(mesh, ("CG", 2))
B4 = dolfinx.fem.Function(W)
B_expr = dolfinx.fem.Expression(ufl.as_vector((A_z4.dx(1), -A_z4.dx(0))), W.element.interpolation_points())
B4.interpolate(B_expr)
multiphenicsx.io.plot_vector_field(B4,"Magnetic Field", glyph_factor=0.1)

## Helmholtz Coil

In [3]:
# Create a model.
gmsh.initialize()
gmsh.model.add("mesh")

# Surface Tags
inner_tag = 2
outer_tag = 2 + N
background_surfaces = []
other_surfaces = []

# Background
background = gmsh.model.occ.addDisk(0, 0, 0, r_background, r_background)
gmsh.model.occ.synchronize()


wires_in = []
wires_out = []

wires_out.append( (2, gmsh.model.occ.addDisk( 1, -0.5, 0, r_wire, r_wire)) )
wires_out.append( (2, gmsh.model.occ.addDisk( 1,  0.5, 0, r_wire, r_wire)) )

wires_in.append( (2, gmsh.model.occ.addDisk(-1, -0.5, 0, r_wire, r_wire)) )
wires_in.append( (2, gmsh.model.occ.addDisk(-1,  0.5, 0, r_wire, r_wire)) )

gmsh.model.occ.synchronize()


# Resolve Boundaries
all_surfaces = []
all_surfaces.extend(wires_in)
all_surfaces.extend(wires_out)
whole_domain = gmsh.model.occ.fragment([(2, background)], all_surfaces)

gmsh.model.occ.synchronize()


# Assign Tags to Surfaces
for domain in whole_domain[0]:
    center = gmsh.model.occ.getCenterOfMass(domain[0], domain[1])
    mass = gmsh.model.occ.getMass(domain[0], domain[1])


    if np.allclose(center, [0, 0, 0]):
        background_surfaces.append(domain[1])

    elif np.isclose(center[0], -1):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], inner_tag)
        inner_tag +=1
        other_surfaces.append(domain)

    elif np.isclose(center[0], +1):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], outer_tag)
        outer_tag +=1
        other_surfaces.append(domain)



gmsh.model.addPhysicalGroup(2, background_surfaces, tag=0)



# Create a mesh for this system.
gmsh.model.mesh.generate(dim)

# Bring the mesh into FEniCSx.
mesh, subdomains, boundaries = dolfinx.io.gmshio.model_to_mesh(
    gmsh.model, comm=mpi4py.MPI.COMM_WORLD, rank=0, gdim=2)

gmsh.finalize()

Q = dolfinx.fem.FunctionSpace(mesh, ("DG", 0))

# Get the list of materials.
material_tags = np.unique(subdomains.values)

# Define functions for current density and magnetic permeability.
mu = dolfinx.fem.Function(Q)
J = dolfinx.fem.Function(Q)

# Only some regions carry current. Initialize all current densities to zero.
J.x.array[:] = 0.0

# Now, cycle over all objects and assign material properties. 
for tag in material_tags:
    cells = subdomains.find(tag)
    
    # Set values for magnetic permeability.
    if tag == 0:
        # Vacuum
        mu_ = mu_background
    else:
        # Wire
        mu_ = mu_wire

    mu.x.array[cells] = np.full_like(cells, mu_, dtype=petsc4py.PETSc.ScalarType)
    
    # Set nonzero current densities.
    if tag in range(2, 2+N):
        J.x.array[cells] = np.full_like(cells, J0, dtype=petsc4py.PETSc.ScalarType)
    elif tag in range(2+N, 2*N + 2):
        J.x.array[cells] = np.full_like(cells, -J0, dtype=petsc4py.PETSc.ScalarType)
        
## Set up the finite element problem.

# Define trial and test functions.
V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 2))

# Define the trial and test functions.
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Create a function to store the solution.
# This is the vector potential.  A_x = A_y = 0.
A_z1 = dolfinx.fem.Function(V)

# Identify the domain and boundary.
D = mesh.topology.dim
Omega = dolfinx.mesh.locate_entities_boundary(mesh, D-1, lambda x: np.full(x.shape[1], True))
dOmega = dolfinx.fem.locate_dofs_topological(V, D-1, Omega)

# Force the potential to vanish on the boundary.
bc = dolfinx.fem.dirichletbc(petsc4py.PETSc.ScalarType(0), dOmega, V)

# Define the Poisson equation we are trying to solve.
a = (1 / mu) * ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = 4 * ufl.pi * J * v * ufl.dx

# Define the problem.
problem = dolfinx.fem.petsc.LinearProblem(a, L, u=A_z1, bcs=[bc])

# Solve the problem.
problem.solve()


# Compute the magnetic field.
W = dolfinx.fem.VectorFunctionSpace(mesh, ("CG", 2))
B1 = dolfinx.fem.Function(W)
B_expr = dolfinx.fem.Expression(ufl.as_vector((A_z1.dx(1), -A_z1.dx(0))), W.element.interpolation_points())
B1.interpolate(B_expr)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 20%] Meshing curve 2 (Ellipse)
Info    : [ 40%] Meshing curve 3 (Ellipse)
Info    : [ 60%] Meshing curve 4 (Ellipse)
Info    : [ 80%] Meshing curve 5 (Ellipse)
Info    : Done meshing 1D (Wall 0.000423583s, CPU 0.000651s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 4 (Plane, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 5 (Plane, Frontal-Delaunay)
Info    : [ 80%] Meshing surface 6 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0210343s, CPU 0.020748s)
Info    : 654 nodes 1339 elements


## 3 Coils

In [4]:
# Create a model.
gmsh.initialize()
gmsh.model.add("mesh")

# Define the system: a large disk.
background = gmsh.model.occ.addDisk(0, 0, 0, r_background, r_background)
gmsh.model.occ.synchronize()

inner_tag = 2
outer_tag = 2 + N
background_surfaces = []
other_surfaces = []

wires_in = []
wires_out = []

wires_out.append( (2, gmsh.model.occ.addDisk( 1, -0.5, 0, r_wire, r_wire)) )
wires_out.append( (2, gmsh.model.occ.addDisk( 1,  0, 0, r_wire, r_wire)) )
wires_out.append( (2, gmsh.model.occ.addDisk( 1,  0.5, 0, r_wire, r_wire)) )

wires_in.append( (2, gmsh.model.occ.addDisk(-1, -0.5, 0, r_wire, r_wire)) )
wires_in.append( (2, gmsh.model.occ.addDisk(-1,  0, 0, r_wire, r_wire)) )
wires_in.append( (2, gmsh.model.occ.addDisk(-1,  0.5, 0, r_wire, r_wire)) )

# Update the model.
gmsh.model.occ.synchronize()

# Resolve the boundaries of the wires and ring in the background domain.

all_surfaces = []
all_surfaces.extend(wires_in)
all_surfaces.extend(wires_out)
whole_domain = gmsh.model.occ.fragment([(2, background)], all_surfaces)

# Update the model.
gmsh.model.occ.synchronize()


inner_tag = 2
outer_tag = 2 + N
background_surfaces = []
other_surfaces = []


for domain in whole_domain[0]:
    center = gmsh.model.occ.getCenterOfMass(domain[0], domain[1])
    mass = gmsh.model.occ.getMass(domain[0], domain[1])

    
    # Identify the background circle by its center of mass
    if np.allclose(center, [0, 0, 0]):
        background_surfaces.append(domain[1])

    # Identify the inner wires by their centers of mass.
    elif np.isclose(center[0], -1):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], inner_tag)
        inner_tag +=1
        other_surfaces.append(domain)

    elif np.isclose(center[0], -1.2):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], inner_tag)
        inner_tag +=1
        other_surfaces.append(domain)

    # Identify the outer wires by their center of mass.
    elif np.isclose(center[0], +1):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], outer_tag)
        outer_tag +=1
        other_surfaces.append(domain)

    elif np.isclose(center[0], +1.2):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], outer_tag)
        outer_tag +=1
        other_surfaces.append(domain)

# Add marker for the vacuum.
gmsh.model.addPhysicalGroup(2, background_surfaces, tag=0)



# Create a mesh for this system.
gmsh.model.mesh.generate(dim)

# Bring the mesh into FEniCSx.
mesh, subdomains, boundaries = dolfinx.io.gmshio.model_to_mesh(
    gmsh.model, comm=mpi4py.MPI.COMM_WORLD, rank=0, gdim=2)

gmsh.finalize()

# This loop will assign material properties to each cell in our model.
# In this case, it is the relative magnetic permeability and current density.

# Define a simple function space for properties.
Q = dolfinx.fem.FunctionSpace(mesh, ("DG", 0))

# Get the list of materials.
material_tags = np.unique(subdomains.values)

# Define functions for current density and magnetic permeability.
mu = dolfinx.fem.Function(Q)
J = dolfinx.fem.Function(Q)

# Only some regions carry current. Initialize all current densities to zero.
J.x.array[:] = 0.0

# Now, cycle over all objects and assign material properties. 
for tag in material_tags:
    cells = subdomains.find(tag)
    
    # Set values for magnetic permeability.
    if tag == 0:
        # Vacuum
        mu_ = mu_background
    elif tag == 1:
        # Ring
        mu_ = mu_ring
    else:
        # Wire
        mu_ = mu_wire

    mu.x.array[cells] = np.full_like(cells, mu_, dtype=petsc4py.PETSc.ScalarType)
    
    # Set nonzero current densities.
    if tag in range(2, 2+N):
        J.x.array[cells] = np.full_like(cells, J0, dtype=petsc4py.PETSc.ScalarType)
    elif tag in range(2+N, 2*N + 2):
        J.x.array[cells] = np.full_like(cells, -J0, dtype=petsc4py.PETSc.ScalarType)
        
## Set up the finite element problem.

# Define trial and test functions.
V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 2))

# Define the trial and test functions.
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Create a function to store the solution.
# This is the vector potential.  A_x = A_y = 0.
A_z2 = dolfinx.fem.Function(V)

# Identify the domain and boundary.
D = mesh.topology.dim
Omega = dolfinx.mesh.locate_entities_boundary(mesh, D-1, lambda x: np.full(x.shape[1], True))
dOmega = dolfinx.fem.locate_dofs_topological(V, D-1, Omega)

# Force the potential to vanish on the boundary.
bc = dolfinx.fem.dirichletbc(petsc4py.PETSc.ScalarType(0), dOmega, V)

# Define the Poisson equation we are trying to solve.
a = (1 / mu) * ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = 4 * ufl.pi * J * v * ufl.dx

# Define the problem.
problem = dolfinx.fem.petsc.LinearProblem(a, L, u=A_z2, bcs=[bc])

# Solve the problem.
problem.solve()


# Compute the magnetic field.
W = dolfinx.fem.VectorFunctionSpace(mesh, ("CG", 2))
B2 = dolfinx.fem.Function(W)
B_expr = dolfinx.fem.Expression(ufl.as_vector((A_z2.dx(1), -A_z2.dx(0))), W.element.interpolation_points())
B2.interpolate(B_expr)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 20%] Meshing curve 2 (Ellipse)
Info    : [ 30%] Meshing curve 3 (Ellipse)
Info    : [ 50%] Meshing curve 4 (Ellipse)
Info    : [ 60%] Meshing curve 5 (Ellipse)
Info    : [ 80%] Meshing curve 6 (Ellipse)
Info    : [ 90%] Meshing curve 7 (Ellipse)
Info    : Done meshing 1D (Wall 0.000736291s, CPU 0.000963s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 4 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 5 (Plane, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 6 (Plane, Frontal-Delaunay)
Info    : [ 80%] Meshing surface 7 (Plane, Frontal-Delaunay)
Info    : [ 90%] Meshing surface 8 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0222415s, CPU 0.022043s)
Info    : 754 nodes 1555 elements


## Maxwell Coil

In [5]:
# Create a model.
gmsh.initialize()
gmsh.model.add("mesh")

# Define the system: a large disk.
background = gmsh.model.occ.addDisk(0, 0, 0, r_background, r_background)
gmsh.model.occ.synchronize()


wires_in = []
wires_out = []

# These seemingly weirdly specific values come from James Clerk Maxwell 
# himself and were outlined in the 1873 book "A Treatise on Electricity and Magentism"
# The design parameters are on pages 320-321
# https://archive.org/details/electricandmag02maxwrich/page/n351/mode/2up?view=theater

wires_out.append( (2, gmsh.model.occ.addDisk( 0.9, -0.786, 0, r_wire, r_wire)) )
wires_out.append( (2, gmsh.model.occ.addDisk( 1.2,  0, 0, r_helm, r_helm)) )
wires_out.append( (2, gmsh.model.occ.addDisk( 0.9,  0.786, 0, r_wire, r_wire)) )

wires_in.append( (2, gmsh.model.occ.addDisk(-0.9, -0.786, 0, r_wire, r_wire)) )
wires_in.append( (2, gmsh.model.occ.addDisk(-1.2,  0, 0, r_helm, r_helm)) )
wires_in.append( (2, gmsh.model.occ.addDisk(-0.9,  0.786, 0, r_wire, r_wire)) )

# Update the model.
gmsh.model.occ.synchronize()

# Resolve the boundaries of the wires and ring in the background domain.

all_surfaces = []
all_surfaces.extend(wires_in)
all_surfaces.extend(wires_out)
whole_domain = gmsh.model.occ.fragment([(2, background)], all_surfaces)

# Update the model.
gmsh.model.occ.synchronize()

# Create physical markers for each object.
# Use the following markers:
# - Vacuum: 0
# - Ring: 1
# - Inner wires: $[2,3,\dots,N+1]$
# - Outer wires: $[N+2,\dots, 2\cdot N+1]$
inner_tag = 2
outer_tag = 2 + N
background_surfaces = []
other_surfaces = []

# Gmsh can compute the mass of objects and the location of their
# centers of mass.  This loop uses these properties to determine
# which object to associate grid points with.
# 
# We will use these tags to define material properties later.
for domain in whole_domain[0]:
    center = gmsh.model.occ.getCenterOfMass(domain[0], domain[1])
    mass = gmsh.model.occ.getMass(domain[0], domain[1])

    
    # Identify the background circle by its center of mass
    if np.allclose(center, [0, 0, 0]):
        background_surfaces.append(domain[1])

    # Identify the inner wires by their centers of mass.
    elif np.isclose(center[0], -0.9):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], inner_tag)
        inner_tag +=1
        other_surfaces.append(domain)

    elif np.isclose(center[0], -1.2):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], inner_tag)
        inner_tag +=1
        other_surfaces.append(domain)

    # Identify the outer wires by their center of mass.
    elif np.isclose(center[0], +0.9):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], outer_tag)
        outer_tag +=1
        other_surfaces.append(domain)

    elif np.isclose(center[0], +1.2):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], outer_tag)
        outer_tag +=1
        other_surfaces.append(domain)

# Add marker for the vacuum.
gmsh.model.addPhysicalGroup(2, background_surfaces, tag=0)



# Create a mesh for this system.
gmsh.model.mesh.generate(dim)

# Bring the mesh into FEniCSx.
mesh, subdomains, boundaries = dolfinx.io.gmshio.model_to_mesh(
    gmsh.model, comm=mpi4py.MPI.COMM_WORLD, rank=0, gdim=2)

gmsh.finalize()

# This loop will assign material properties to each cell in our model.
# In this case, it is the relative magnetic permeability and current density.

# Define a simple function space for properties.
Q = dolfinx.fem.FunctionSpace(mesh, ("DG", 0))

# Get the list of materials.
material_tags = np.unique(subdomains.values)

# Define functions for current density and magnetic permeability.
mu = dolfinx.fem.Function(Q)
J = dolfinx.fem.Function(Q)

# Only some regions carry current. Initialize all current densities to zero.
J.x.array[:] = 0.0

# Now, cycle over all objects and assign material properties. 
for tag in material_tags:
    cells = subdomains.find(tag)
    
    # Set values for magnetic permeability.
    if tag == 0:
        # Vacuum
        mu_ = mu_background

    else:
        # Wire
        mu_ = mu_wire

    mu.x.array[cells] = np.full_like(cells, mu_, dtype=petsc4py.PETSc.ScalarType)
    
    # Set nonzero current densities.
    if tag in range(2, 2+N):
        J.x.array[cells] = np.full_like(cells, J0, dtype=petsc4py.PETSc.ScalarType)
    elif tag in range(2+N, 2*N + 2):
        J.x.array[cells] = np.full_like(cells, -J0, dtype=petsc4py.PETSc.ScalarType)
        
## Set up the finite element problem.

# Define trial and test functions.
V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 2))

# Define the trial and test functions.
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Create a function to store the solution.
# This is the vector potential.  A_x = A_y = 0.
A_z3 = dolfinx.fem.Function(V)

# Identify the domain and boundary.
D = mesh.topology.dim
Omega = dolfinx.mesh.locate_entities_boundary(mesh, D-1, lambda x: np.full(x.shape[1], True))
dOmega = dolfinx.fem.locate_dofs_topological(V, D-1, Omega)

# Force the potential to vanish on the boundary.
bc = dolfinx.fem.dirichletbc(petsc4py.PETSc.ScalarType(0), dOmega, V)

# Define the Poisson equation we are trying to solve.
a = (1 / mu) * ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = 4 * ufl.pi * J * v * ufl.dx

# Define the problem.
problem = dolfinx.fem.petsc.LinearProblem(a, L, u=A_z3, bcs=[bc])

# Solve the problem.
problem.solve()


# Compute the magnetic field.
W = dolfinx.fem.VectorFunctionSpace(mesh, ("CG", 2))
B3 = dolfinx.fem.Function(W)
B_expr = dolfinx.fem.Expression(ufl.as_vector((A_z3.dx(1), -A_z3.dx(0))), W.element.interpolation_points())
B3.interpolate(B_expr)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 20%] Meshing curve 2 (Ellipse)
Info    : [ 30%] Meshing curve 3 (Ellipse)
Info    : [ 50%] Meshing curve 4 (Ellipse)
Info    : [ 60%] Meshing curve 5 (Ellipse)
Info    : [ 80%] Meshing curve 6 (Ellipse)
Info    : [ 90%] Meshing curve 7 (Ellipse)
Info    : Done meshing 1D (Wall 0.000978125s, CPU 0.001096s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 4 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 5 (Plane, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 6 (Plane, Frontal-Delaunay)
Info    : [ 80%] Meshing surface 7 (Plane, Frontal-Delaunay)
Info    : [ 90%] Meshing surface 8 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0263374s, CPU 0.026107s)
Info    : 911 nodes 1869 elements


## Nothing Particularly Identifiable

In [6]:
# Create a model.
gmsh.initialize()
gmsh.model.add("mesh")

# Define the system: a large disk.
background = gmsh.model.occ.addDisk(0, 0, 0, r_background, r_background)
gmsh.model.occ.synchronize()

inner_tag = 2
outer_tag = 2 + N
background_surfaces = []
other_surfaces = []

wires_in = []
wires_out = []

wires_out.append( (2, gmsh.model.occ.addDisk( 1, -0.5, 0, r_wire, r_wire)))
wires_out.append( (2, gmsh.model.occ.addDisk( 1.2,  -0.25, 0, r_helm, r_helm)))
wires_out.append( (2, gmsh.model.occ.addDisk( 1.4,  0, 0, r_wider, r_wider)))
wires_out.append( (2, gmsh.model.occ.addDisk( 1.2,  0.25, 0, r_helm, r_helm)))
wires_out.append( (2, gmsh.model.occ.addDisk( 1,  0.5, 0, r_wire, r_wire)))

wires_in.append( (2, gmsh.model.occ.addDisk(-1, -0.5, 0, r_wire, r_wire)))
wires_in.append( (2, gmsh.model.occ.addDisk(-1.2,  -0.25, 0, r_helm, r_helm)))
wires_in.append( (2, gmsh.model.occ.addDisk(-1.4,  0, 0, r_wider, r_wider)))
wires_in.append( (2, gmsh.model.occ.addDisk(-1.2,  0.25, 0, r_helm, r_helm)))
wires_in.append( (2, gmsh.model.occ.addDisk(-1,  0.5, 0, r_wire, r_wire)))

# Update the model.
gmsh.model.occ.synchronize()

# Resolve the boundaries of the wires and ring in the background domain.

all_surfaces = []
all_surfaces.extend(wires_in)
all_surfaces.extend(wires_out)
whole_domain = gmsh.model.occ.fragment([(2, background)], all_surfaces)

# Update the model.
gmsh.model.occ.synchronize()


for domain in whole_domain[0]:
    center = gmsh.model.occ.getCenterOfMass(domain[0], domain[1])
    mass = gmsh.model.occ.getMass(domain[0], domain[1])

    
    # Identify the background circle by its center of mass
    if np.allclose(center, [0, 0, 0]):
        background_surfaces.append(domain[1])

    # Identify the inner wires by their centers of mass.
    elif np.isclose(center[0], -1):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], inner_tag)
        inner_tag +=1
        other_surfaces.append(domain)

    elif np.isclose(center[0], -1.2):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], inner_tag)
        inner_tag +=1
        other_surfaces.append(domain)

    elif np.isclose(center[0], -1.4):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], inner_tag)
        inner_tag +=1
        other_surfaces.append(domain)

    # Identify the outer wires by their center of mass.
    elif np.isclose(center[0], +1):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], outer_tag)
        outer_tag +=1
        other_surfaces.append(domain)

    elif np.isclose(center[0], +1.2):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], outer_tag)
        outer_tag +=1
        other_surfaces.append(domain)


    elif np.isclose(center[0], +1.4):
        gmsh.model.addPhysicalGroup(domain[0], [domain[1]], outer_tag)
        outer_tag +=1
        other_surfaces.append(domain)



        




# Add marker for the vacuum.
gmsh.model.addPhysicalGroup(2, background_surfaces, tag=0)



# Create a mesh for this system.
gmsh.model.mesh.generate(dim)

# Bring the mesh into FEniCSx.
mesh, subdomains, boundaries = dolfinx.io.gmshio.model_to_mesh(
    gmsh.model, comm=mpi4py.MPI.COMM_WORLD, rank=0, gdim=2)

gmsh.finalize()

# This loop will assign material properties to each cell in our model.
# In this case, it is the relative magnetic permeability and current density.

# Define a simple function space for properties.
Q = dolfinx.fem.FunctionSpace(mesh, ("DG", 0))

# Get the list of materials.
material_tags = np.unique(subdomains.values)

# Define functions for current density and magnetic permeability.
mu = dolfinx.fem.Function(Q)
J = dolfinx.fem.Function(Q)

# Only some regions carry current. Initialize all current densities to zero.
J.x.array[:] = 0.0

# Now, cycle over all objects and assign material properties. 
for tag in material_tags:
    cells = subdomains.find(tag)
    
    # Set values for magnetic permeability.
    if tag == 0:
        # Vacuum
        mu_ = mu_background

    else:
        # Wire
        mu_ = mu_wire

    mu.x.array[cells] = np.full_like(cells, mu_, dtype=petsc4py.PETSc.ScalarType)
    
    # Set nonzero current densities.
    if tag in range(2, 2+N):
        J.x.array[cells] = np.full_like(cells, J0, dtype=petsc4py.PETSc.ScalarType)
    elif tag in range(2+N, 2*N + 2):
        J.x.array[cells] = np.full_like(cells, -J0, dtype=petsc4py.PETSc.ScalarType)
        
## Set up the finite element problem.

# Define trial and test functions.
V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 2))

# Define the trial and test functions.
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Create a function to store the solution.
# This is the vector potential.  A_x = A_y = 0.
A_z4 = dolfinx.fem.Function(V)

# Identify the domain and boundary.
D = mesh.topology.dim
Omega = dolfinx.mesh.locate_entities_boundary(mesh, D-1, lambda x: np.full(x.shape[1], True))
dOmega = dolfinx.fem.locate_dofs_topological(V, D-1, Omega)

# Force the potential to vanish on the boundary.
bc = dolfinx.fem.dirichletbc(petsc4py.PETSc.ScalarType(0), dOmega, V)

# Define the Poisson equation we are trying to solve.
a = (1 / mu) * ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = 4 * ufl.pi * J * v * ufl.dx

# Define the problem.
problem = dolfinx.fem.petsc.LinearProblem(a, L, u=A_z4, bcs=[bc])

# Solve the problem.
problem.solve()


# Compute the magnetic field.
W = dolfinx.fem.VectorFunctionSpace(mesh, ("CG", 2))
B4 = dolfinx.fem.Function(W)
B_expr = dolfinx.fem.Expression(ufl.as_vector((A_z4.dx(1), -A_z4.dx(0))), W.element.interpolation_points())
B4.interpolate(B_expr)
multiphenicsx.io.plot_vector_field(B4,"Magnetic Field", glyph_factor=0.1)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 10%] Meshing curve 2 (Ellipse)
Info    : [ 20%] Meshing curve 3 (Ellipse)
Info    : [ 30%] Meshing curve 4 (Ellipse)
Info    : [ 40%] Meshing curve 5 (Ellipse)
Info    : [ 50%] Meshing curve 6 (Ellipse)
Info    : [ 60%] Meshing curve 7 (Ellipse)
Info    : [ 70%] Meshing curve 8 (Ellipse)
Info    : [ 80%] Meshing curve 9 (Ellipse)
Info    : [ 90%] Meshing curve 10 (Ellipse)
Info    : [100%] Meshing curve 11 (Ellipse)
Info    : Done meshing 1D (Wall 0.00108338s, CPU 0.001344s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 10%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 4 (Plane, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 5 (Plane, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 6 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 7 (Plane, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 8 (Plane, Frontal-

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

# Simulations

## Standard 2 Coil Helmholtz

In [7]:
multiphenicsx.io.plot_scalar_field(A_z1,"Vector Potential", warp_factor=1)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [8]:
multiphenicsx.io.plot_vector_field(B1,"Magnetic Field", glyph_factor=0.1)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## 3 Coil 

In [9]:
multiphenicsx.io.plot_scalar_field(A_z2,"Vector Potential", warp_factor=1)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [10]:
multiphenicsx.io.plot_vector_field(B2,"Magnetic Field", glyph_factor=0.1)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Maxwell Coil

In [11]:
multiphenicsx.io.plot_scalar_field(A_z3,"Vector Potential", warp_factor=1)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [12]:
multiphenicsx.io.plot_vector_field(B3,"Magnetic Field", glyph_factor=0.1)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Helmholtz-Helmholtz-Maxwell Coil

In [13]:
multiphenicsx.io.plot_scalar_field(A_z4,"Vector Potential", warp_factor=1)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [14]:
multiphenicsx.io.plot_vector_field(B4,"Magnetic Field", glyph_factor=0.1)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)